In [106]:
#Step 1: Setup and Installation
!pip install google-cloud  # Install the required Google Cloud libraries

In [108]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=AAUXvqqw7ibJBSltM18LiNFXwWZJbg&prompt=consent&token_usage=remote&access_type=offline&code_challenge=yCVOWI2Q6J0E1TZwFh71FiAt8eHTsHTkeV6c1F8wNGY&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0ASVgi3J6B80iSGa9Kq1p7-VKB5LHZ8YkkIql9kOA_0KNsmuvl3C-d5RpHqn--l29viWNMA
ERROR: There was a problem with web authentication.
ERROR: (gcloud.auth.application-default.login) (invalid_grant) Bad Request


In [111]:
#Step 2: Load and Preprocess Data
import pandas as pd

# Example of loading the dataframe (assuming you already have it loaded)
train_df = pd.read_csv("/content/sample_data/train_5v5GIB2.csv")
test_df = pd.read_csv("/content/sample_data/test_sample.csv")

# Preprocessing function with NaN check
def preprocess_text(text):
    if pd.isna(text):  # Check if the text is NaN or None
        return ""
    text = str(text)  # Ensure the text is a string
    text = text.lower()  # Convert to lowercase
    text = ' '.join(text.split())  # Remove extra spaces
    return text

# Apply preprocessing to the subject and body of tickets
train_df['ticket_subject'] = train_df['ticket_subject'].apply(preprocess_text)
train_df['ticket_body'] = train_df['ticket_body'].apply(preprocess_text)

# Similarly preprocess test data
test_df['ticket_subject'] = test_df['ticket_subject'].apply(preprocess_text)
test_df['ticket_body'] = test_df['ticket_body'].apply(preprocess_text)

# Preview the preprocessed data
print(train_df.head())

   ticket_ID                                     ticket_subject  \
0       1001    discrepancia de facturación en google workspace   
1       1002  urgent consultation request for critical it is...   
2       1003      consulta sobre servicios de consultoría en ti   
3       1004                demande de mise à jour des dossiers   
4       1005  issues with slack connection affecting team co...   

                                         ticket_body            department  \
0  estimado equipo de soporte de ti, estoy escrib...  Billing and Payments   
1  dear it services support team, i hope this mes...      Customer Service   
2  estimado servicio de atención al cliente, mi n...       General Inquiry   
3  cher service client, je vous écris pour demand...       Human Resources   
4  dear customer support team, i am encountering ...       Product Support   

       type priority language  
0  Incident      low       es  
1   Request     high       en  
2   Request   medium       es  


In [95]:
#Step 3: Define Prompts for Google Gemini
def generate_department_prompt(ticket_subject, ticket_body):
    prompt = f"""
    You are a support agent. Please classify the following IT support ticket based on its subject and body.

    Ticket Subject: {ticket_subject}
    Ticket Body: {ticket_body}

    The department categories are:
    - Technical Support
    - Customer Service
    - Billing and Payments
    - Product Support
    - IT Support
    - Returns and Exchanges
    - Sales and Pre-Sales
    - Human Resources
    - Service Outages and Maintenance
    - General Inquiry

    Please provide the department to which this ticket should be routed.
    """
    return prompt

# For Priority
def generate_priority_prompt(ticket_subject, ticket_body):
    prompt = f"""
    Please classify the urgency of the following ticket.

    Ticket Subject: {ticket_subject}
    Ticket Body: {ticket_body}

    The urgency levels are:
    - Low
    - Medium
    - High

    Please classify the priority level.
    """
    return prompt

# For Language
def generate_language_prompt(ticket_subject, ticket_body):
    prompt = f"""
    Please identify the language of the following ticket.

    Ticket Subject: {ticket_subject}
    Ticket Body: {ticket_body}

    Please return the language code (e.g., 'en' for English, 'de' for German, etc.).
    """
    return prompt

# For Type
def generate_type_prompt(ticket_subject, ticket_body):
    prompt = f"""
    Please classify the type of the following ticket.

    Ticket Subject: {ticket_subject}
    Ticket Body: {ticket_body}

    The ticket types are:
    - Incident
    - Request
    - Problem
    - Change

    Please classify the type of the ticket.
    """
    return prompt

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [96]:
#Step 4: Make Predictions Using Google Gemini
import openai  # Example of using OpenAI's GPT, but you would use the Gemini API if available

# Set up the API key or endpoint for Google Gemini
openai.api_key = 'your-google-gemini-api-key'

# Function to query the model with a prompt
def get_prediction_from_gemini(prompt):
    response = openai.Completion.create(
        model="gpt-4",  # Replace with Google Gemini model ID if available
        prompt=prompt,
        max_tokens=50
    )
    return response.choices[0].text.strip()

In [97]:
#Step 5: Apply the Model to the Test Dataset
# Initialize result containers
predicted_departments = []
predicted_priorities = []
predicted_languages = []
predicted_types = []

# Loop through the test data and make predictions
for _, row in test_df.iterrows():
    subject = row['ticket_subject']
    body = row['ticket_body']

    # Generate prompts
    department_prompt = generate_department_prompt(subject, body)
    priority_prompt = generate_priority_prompt(subject, body)
    language_prompt = generate_language_prompt(subject, body)
    type_prompt = generate_type_prompt(subject, body)

    # Get predictions
    department = get_prediction_from_gemini(department_prompt)
    priority = get_prediction_from_gemini(priority_prompt)
    language = get_prediction_from_gemini(language_prompt)
    ticket_type = get_prediction_from_gemini(type_prompt)

    # Store predictions
    predicted_departments.append(department)
    predicted_priorities.append(priority)
    predicted_languages.append(language)
    predicted_types.append(ticket_type)

# Create a results dataframe
results_df = test_df.copy()
results_df['predicted_department'] = predicted_departments
results_df['predicted_priority'] = predicted_priorities
results_df['predicted_language'] = predicted_languages
results_df['predicted_type'] = predicted_types

# Save results to CSV
results_df.to_csv("predictions.csv", index=False)

In [98]:
#Step 6: Evaluation
from sklearn.metrics import accuracy_score

# Assuming you have the true labels in the test dataset (e.g., 'department', 'priority', etc.)
department_accuracy = accuracy_score(test_df['department'], predicted_departments)
priority_accuracy = accuracy_score(test_df['priority'], predicted_priorities)
language_accuracy = accuracy_score(test_df['language'], predicted_languages)
type_accuracy = accuracy_score(test_df['type'], predicted_types)

# Calculate final weighted score
final_score = (0.4 * department_accuracy) + (0.3 * type_accuracy) + (0.1 * language_accuracy) + (0.2 * priority_accuracy)

print(f"Department Accuracy: {department_accuracy:.4f}")
print(f"Priority Accuracy: {priority_accuracy:.4f}")
print(f"Language Accuracy: {language_accuracy:.4f}")
print(f"Type Accuracy: {type_accuracy:.4f}")
print(f"Final Weighted Score: {final_score:.4f}")